Count number of FTUs in masks

In [ ]:
import numpy as np
import pandas as pd 
import json
import cv2
import codecs
import os
import skimage.io as io

In [ ]:
path_to_json = "path/to/json"
path_to_metadata = "path/to/metadata.csv"

In [ ]:
metadata = pd.read_csv(path_to_metadata)

In [ ]:
metadata

In [ ]:
fnames = []
organs = []
ftus = []
skp = []
for ann in os.listdir(path_to_json):
    for j in os.listdir(path_to_json+"/"+ann):
        json_path = path_to_json+"/"+ann+"/"+j
    
        data = json.load(codecs.open(json_path, 'r', 'utf-8-sig'))
        
        fname = j.split(".")[0]

        polys = []
        ftu_count = 0
        skipped = 0
        for index in range(data.__len__()):
            geom = [np.array([x]) for x in data[index]['geometry']['coordinates']]

            for g in geom:
                if cv2.contourArea(g.astype(int)) >= 10:
                    ftu_count+=1
                else:
                    skipped+=1

        ftus.append(ftu_count)
        skp.append(skipped)
        fnames.append(fname)
        organs.append(ann)

In [ ]:
df = pd.DataFrame(ftus,fnames).reset_index()
df.columns = ["filename","ftu_count"]

In [ ]:
def get_ftu(x):
    if x in df['filename'].tolist():
        return df[df['filename']==x]['ftu_count'].values[0]
    else:
        return 0

In [ ]:
metadata["ftu_counts"]  = metadata['filename'].apply(lambda x : get_ftu(x))